# Ekstraksi Informasi dari Berita Kriminal Indonesia menggunakan Named Entity Recognition (NER).

## Setting Up
### Install the required libraries

In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory as SastrawiStopWordRemoverFactory
nltk.download('punkt')  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fairuuz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load Dataset

In [2]:
df = pd.read_csv("../data/processed/cleaned_all_data.csv", sep=',')
df.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764
2,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,2024-09-03 08:01:00,Hidup pria bernama Monika Hutauruk (45) harus ...,68,3907


## Preprocessing
### Tokenization

In [ ]:
def remove_punctuation(text):
    if not isinstance(text, str):
        text = str(text)
    return re.sub(r'[^\w\s]', '', text)

df['judul_rp'] = df['judul'].apply(remove_punctuation)
df['isi_rp'] = df['isi_berita'].apply(remove_punctuation)

print("Hasil setelah membersihkan tanda baca secara terpisah:")
print(df[['judul', 'judul_rp', 'isi_berita', 'isi_rp']].head())

Hasil setelah membersihkan tanda baca secara terpisah:
                                               judul  \
0  4 Anak di Bawah Umur Tersangka Pembunuhan di P...   
1  26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...   
2  Pekerja Kafe Disiram Air Keras hingga Wajah 'B...   
3  Hilang Nyawa Pria di Simalungun gegara Rebutan...   
4  Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...   

                                            judul_rp  \
0  4 Anak di Bawah Umur Tersangka Pembunuhan di P...   
1  26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...   
2  Pekerja Kafe Disiram Air Keras hingga Wajah Be...   
3  Hilang Nyawa Pria di Simalungun gegara Rebutan...   
4  Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...   

                                          isi_berita  \
0  Pelaku pembunuhan dan pemerkosaan AA (14) seor...   
1  Dalam waktu kurang lebih 3 bulan, Polres Probo...   
2  Seorang pekerja kafe di Cengkareng, MAS (32), ...   
3  Hanya gegara rebutan mikrofon untuk bernyany

In [4]:
df['judul_rp'] = df['judul_rp'].fillna('')
df['isi_rp'] = df['isi_rp'].fillna('')

# Tokenisasi 
df['judul_tokens'] = df['judul_rp'].apply(word_tokenize)
df['isi_tokens'] = df['isi_rp'].apply(word_tokenize)

# Hasil tokenisasi
df[['judul_tokens', 'isi_tokens']].head()

,judul_tokens,isi_tokens
0,"[4, Anak, di, Bawah, Umur, Tersangka, Pembunuh...","[Pelaku, pembunuhan, dan, pemerkosaan, AA, 14,..."
1,"[26, Tersangka, Dibekuk, Selama, 3, Bulan, Ter...","[Dalam, waktu, kurang, lebih, 3, bulan, Polres..."
2,"[Pekerja, Kafe, Disiram, Air, Keras, hingga, W...","[Seorang, pekerja, kafe, di, Cengkareng, MAS, ..."
3,"[Hilang, Nyawa, Pria, di, Simalungun, gegara, ...","[Hanya, gegara, rebutan, mikrofon, untuk, bern..."
4,"[Utang, Rp, 3, Juta, Bikin, Pegawai, Akper, Te...","[Hidup, pria, bernama, Monika, Hutauruk, 45, h..."


### Casefolding


In [5]:
df['judul_tokens_lower'] = df['judul_tokens'].apply(lambda tokens: [token.lower() for token in tokens])
df['isi_tokens_lower'] = df['isi_tokens'].apply(lambda tokens: [token.lower() for token in tokens])

df[['judul_tokens_lower', 'isi_tokens_lower']].head()

,judul_tokens_lower,isi_tokens_lower
0,"[4, anak, di, bawah, umur, tersangka, pembunuh...","[pelaku, pembunuhan, dan, pemerkosaan, aa, 14,..."
1,"[26, tersangka, dibekuk, selama, 3, bulan, ter...","[dalam, waktu, kurang, lebih, 3, bulan, polres..."
2,"[pekerja, kafe, disiram, air, keras, hingga, w...","[seorang, pekerja, kafe, di, cengkareng, mas, ..."
3,"[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebutan, mikrofon, untuk, bern..."
4,"[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, bernama, monika, hutauruk, 45, h..."


### Stemming

In [6]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

df['judul_stemmed'] = df['judul_tokens_lower'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
df['isi_stemmed'] = df['isi_tokens_lower'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

df[['judul_stemmed', 'isi_stemmed']].head()

,judul_stemmed,isi_stemmed
0,"[4, anak, di, bawah, umur, sangka, bunuh, di, ...","[laku, bunuh, dan, perkosa, aa, 14, orang, rem..."
1,"[26, sangka, bekuk, lama, 3, bulan, akhir, di,...","[dalam, waktu, kurang, lebih, 3, bulan, polres..."
2,"[kerja, kafe, siram, air, keras, hingga, wajah...","[orang, kerja, kafe, di, cengkareng, mas, 32, ..."
3,"[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebut, mikrofon, untuk, nyanyi..."
4,"[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, nama, monika, hutauruk, 45, haru..."


In [7]:
df.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita,judul_rp,isi_rp,judul_tokens,isi_tokens,judul_tokens_lower,isi_tokens_lower,judul_stemmed,isi_stemmed
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,Pelaku pembunuhan dan pemerkosaan AA 14 seoran...,"[4, Anak, di, Bawah, Umur, Tersangka, Pembunuh...","[Pelaku, pembunuhan, dan, pemerkosaan, AA, 14,...","[4, anak, di, bawah, umur, tersangka, pembunuh...","[pelaku, pembunuhan, dan, pemerkosaan, aa, 14,...","[4, anak, di, bawah, umur, sangka, bunuh, di, ...","[laku, bunuh, dan, perkosa, aa, 14, orang, rem..."
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,Dalam waktu kurang lebih 3 bulan Polres Probol...,"[26, Tersangka, Dibekuk, Selama, 3, Bulan, Ter...","[Dalam, waktu, kurang, lebih, 3, bulan, Polres...","[26, tersangka, dibekuk, selama, 3, bulan, ter...","[dalam, waktu, kurang, lebih, 3, bulan, polres...","[26, sangka, bekuk, lama, 3, bulan, akhir, di,...","[dalam, waktu, kurang, lebih, 3, bulan, polres..."
2,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995,Pekerja Kafe Disiram Air Keras hingga Wajah Be...,Seorang pekerja kafe di Cengkareng MAS 32 didu...,"[Pekerja, Kafe, Disiram, Air, Keras, hingga, W...","[Seorang, pekerja, kafe, di, Cengkareng, MAS, ...","[pekerja, kafe, disiram, air, keras, hingga, w...","[seorang, pekerja, kafe, di, cengkareng, mas, ...","[kerja, kafe, siram, air, keras, hingga, wajah...","[orang, kerja, kafe, di, cengkareng, mas, 32, ..."
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398,Hilang Nyawa Pria di Simalungun gegara Rebutan...,Hanya gegara rebutan mikrofon untuk bernyanyi ...,"[Hilang, Nyawa, Pria, di, Simalungun, gegara, ...","[Hanya, gegara, rebutan, mikrofon, untuk, bern...","[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebutan, mikrofon, untuk, bern...","[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebut, mikrofon, untuk, nyanyi..."
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,2024-09-03 08:01:00,Hidup pria bernama Monika Hutauruk (45) harus ...,68,3907,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,Hidup pria bernama Monika Hutauruk 45 harus be...,"[Utang, Rp, 3, Juta, Bikin, Pegawai, Akper, Te...","[Hidup, pria, bernama, Monika, Hutauruk, 45, h...","[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, bernama, monika, hutauruk, 45, h...","[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, nama, monika, hutauruk, 45, haru..."


In [ ]:
df.to_csv("../data/processed/stemmed.csv", sep=',', index=False)

### Stop Word Removal & POS Tagging

In [2]:
df = pd.read_csv("../data/processed/stemmed.csv", sep=',')
df.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita,judul_rp,isi_rp,judul_tokens,isi_tokens,judul_tokens_lower,isi_tokens_lower,judul_stemmed,isi_stemmed
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,Pelaku pembunuhan dan pemerkosaan AA 14 seoran...,"['4', 'Anak', 'di', 'Bawah', 'Umur', 'Tersangk...","['Pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","['4', 'anak', 'di', 'bawah', 'umur', 'tersangk...","['pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","['4', 'anak', 'di', 'bawah', 'umur', 'sangka',...","['laku', 'bunuh', 'dan', 'perkosa', 'aa', '14'..."
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,Dalam waktu kurang lebih 3 bulan Polres Probol...,"['26', 'Tersangka', 'Dibekuk', 'Selama', '3', ...","['Dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['26', 'tersangka', 'dibekuk', 'selama', '3', ...","['dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['26', 'sangka', 'bekuk', 'lama', '3', 'bulan'...","['dalam', 'waktu', 'kurang', 'lebih', '3', 'bu..."
2,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995,Pekerja Kafe Disiram Air Keras hingga Wajah Be...,Seorang pekerja kafe di Cengkareng MAS 32 didu...,"['Pekerja', 'Kafe', 'Disiram', 'Air', 'Keras',...","['Seorang', 'pekerja', 'kafe', 'di', 'Cengkare...","['pekerja', 'kafe', 'disiram', 'air', 'keras',...","['seorang', 'pekerja', 'kafe', 'di', 'cengkare...","['kerja', 'kafe', 'siram', 'air', 'keras', 'hi...","['orang', 'kerja', 'kafe', 'di', 'cengkareng',..."
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398,Hilang Nyawa Pria di Simalungun gegara Rebutan...,Hanya gegara rebutan mikrofon untuk bernyanyi ...,"['Hilang', 'Nyawa', 'Pria', 'di', 'Simalungun'...","['Hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","['hilang', 'nyawa', 'pria', 'di', 'simalungun'...","['hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","['hilang', 'nyawa', 'pria', 'di', 'simalungun'...","['hanya', 'gegara', 'rebut', 'mikrofon', 'untu..."
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,2024-09-03 08:01:00,Hidup pria bernama Monika Hutauruk (45) harus ...,68,3907,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,Hidup pria bernama Monika Hutauruk 45 harus be...,"['Utang', 'Rp', '3', 'Juta', 'Bikin', 'Pegawai...","['Hidup', 'pria', 'bernama', 'Monika', 'Hutaur...","['utang', 'rp', '3', 'juta', 'bikin', 'pegawai...","['hidup', 'pria', 'bernama', 'monika', 'hutaur...","['utang', 'rp', '3', 'juta', 'bikin', 'pegawai...","['hidup', 'pria', 'nama', 'monika', 'hutauruk'..."


In [3]:
import ast

def convert_string_to_list(text_list):
    try:
        if isinstance(text_list, str) and text_list.startswith('[') and text_list.endswith(']'):
            return ast.literal_eval(text_list)
        elif isinstance(text_list, list): 
            return text_list
    except (ValueError, SyntaxError):
        print(f"Peringatan: Gagal mengonversi string ke list: {text_list}")
        return []
    return [] 

for col in ['judul_stemmed', 'isi_stemmed']:
    if col in df.columns:
        df[col] = df[col].apply(convert_string_to_list)
    else:
        print(f"Kolom {col} tidak ditemukan.")

print("Tipe data setelah konversi dan contoh isi kolom stemmed:")
if 'judul_stemmed' in df.columns and not df['judul_stemmed'].empty:
    print(f"Tipe judul_stemmed[0]: {type(df['judul_stemmed'].iloc[0])}")
    print(df[['judul_stemmed', 'isi_stemmed']].head())

Tipe data setelah konversi dan contoh isi kolom stemmed:
Tipe judul_stemmed[0]: <class 'list'>
                                       judul_stemmed  \
0  [4, anak, di, bawah, umur, sangka, bunuh, di, ...   
1  [26, sangka, bekuk, lama, 3, bulan, akhir, di,...   
2  [kerja, kafe, siram, air, keras, hingga, wajah...   
3  [hilang, nyawa, pria, di, simalungun, gegara, ...   
4  [utang, rp, 3, juta, bikin, pegawai, akper, te...   

                                         isi_stemmed  
0  [laku, bunuh, dan, perkosa, aa, 14, orang, rem...  
1  [dalam, waktu, kurang, lebih, 3, bulan, polres...  
2  [orang, kerja, kafe, di, cengkareng, mas, 32, ...  
3  [hanya, gegara, rebut, mikrofon, untuk, nyanyi...  
4  [hidup, pria, nama, monika, hutauruk, 45, haru...  


In [4]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory as SastrawiStopWordRemoverFactory

sastrawi_factory = SastrawiStopWordRemoverFactory()
sastrawi_stopwords_list = sastrawi_factory.get_stop_words()
print(f"Contoh beberapa stopwords Sastrawi: {sastrawi_stopwords_list[:10]}")

def remove_sastrawi_stopwords(tokens):
    if not isinstance(tokens, list):
        return [] 
    return [token for token in tokens if token not in sastrawi_stopwords_list]

df['judul_stop_removed'] = df['judul_stemmed'].apply(remove_sastrawi_stopwords)
df['isi_stop_removed'] = df['isi_stemmed'].apply(remove_sastrawi_stopwords)

print("\nDataFrame setelah Stopword Removal:")
df[['judul_stemmed', 'judul_stop_removed', 'isi_stemmed', 'isi_stop_removed']].head()

Contoh beberapa stopwords Sastrawi: ['yang', 'untuk', 'pada', 'ke', 'para', 'namun', 'menurut', 'antara', 'dia', 'dua']

DataFrame setelah Stopword Removal:


,judul_stemmed,judul_stop_removed,isi_stemmed,isi_stop_removed
0,"[4, anak, di, bawah, umur, sangka, bunuh, di, ...","[4, anak, bawah, umur, sangka, bunuh, palemban...","[laku, bunuh, dan, perkosa, aa, 14, orang, rem...","[laku, bunuh, perkosa, aa, 14, orang, remaja, ..."
1,"[26, sangka, bekuk, lama, 3, bulan, akhir, di,...","[26, sangka, bekuk, lama, 3, bulan, akhir, kot...","[dalam, waktu, kurang, lebih, 3, bulan, polres...","[waktu, kurang, lebih, 3, bulan, polres, probo..."
2,"[kerja, kafe, siram, air, keras, hingga, wajah...","[kerja, kafe, siram, air, keras, hingga, wajah...","[orang, kerja, kafe, di, cengkareng, mas, 32, ...","[orang, kerja, kafe, cengkareng, mas, 32, duga..."
3,"[hilang, nyawa, pria, di, simalungun, gegara, ...","[hilang, nyawa, pria, simalungun, gegara, rebu...","[hanya, gegara, rebut, mikrofon, untuk, nyanyi...","[gegara, rebut, mikrofon, nyanyi, warung, tuak..."
4,"[utang, rp, 3, juta, bikin, pegawai, akper, te...","[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, nama, monika, hutauruk, 45, haru...","[hidup, pria, nama, monika, hutauruk, 45, akhi..."


In [5]:
df.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita,judul_rp,isi_rp,judul_tokens,isi_tokens,judul_tokens_lower,isi_tokens_lower,judul_stemmed,isi_stemmed,judul_stop_removed,isi_stop_removed
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,Pelaku pembunuhan dan pemerkosaan AA 14 seoran...,"['4', 'Anak', 'di', 'Bawah', 'Umur', 'Tersangk...","['Pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","['4', 'anak', 'di', 'bawah', 'umur', 'tersangk...","['pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","[4, anak, di, bawah, umur, sangka, bunuh, di, ...","[laku, bunuh, dan, perkosa, aa, 14, orang, rem...","[4, anak, bawah, umur, sangka, bunuh, palemban...","[laku, bunuh, perkosa, aa, 14, orang, remaja, ..."
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,Dalam waktu kurang lebih 3 bulan Polres Probol...,"['26', 'Tersangka', 'Dibekuk', 'Selama', '3', ...","['Dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['26', 'tersangka', 'dibekuk', 'selama', '3', ...","['dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","[26, sangka, bekuk, lama, 3, bulan, akhir, di,...","[dalam, waktu, kurang, lebih, 3, bulan, polres...","[26, sangka, bekuk, lama, 3, bulan, akhir, kot...","[waktu, kurang, lebih, 3, bulan, polres, probo..."
2,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995,Pekerja Kafe Disiram Air Keras hingga Wajah Be...,Seorang pekerja kafe di Cengkareng MAS 32 didu...,"['Pekerja', 'Kafe', 'Disiram', 'Air', 'Keras',...","['Seorang', 'pekerja', 'kafe', 'di', 'Cengkare...","['pekerja', 'kafe', 'disiram', 'air', 'keras',...","['seorang', 'pekerja', 'kafe', 'di', 'cengkare...","[kerja, kafe, siram, air, keras, hingga, wajah...","[orang, kerja, kafe, di, cengkareng, mas, 32, ...","[kerja, kafe, siram, air, keras, hingga, wajah...","[orang, kerja, kafe, cengkareng, mas, 32, duga..."
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398,Hilang Nyawa Pria di Simalungun gegara Rebutan...,Hanya gegara rebutan mikrofon untuk bernyanyi ...,"['Hilang', 'Nyawa', 'Pria', 'di', 'Simalungun'...","['Hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","['hilang', 'nyawa', 'pria', 'di', 'simalungun'...","['hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebut, mikrofon, untuk, nyanyi...","[hilang, nyawa, pria, simalungun, gegara, rebu...","[gegara, rebut, mikrofon, nyanyi, warung, tuak..."
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,2024-09-03 08:01:00,Hidup pria bernama Monika Hutauruk (45) harus ...,68,3907,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,Hidup pria bernama Monika Hutauruk 45 harus be...,"['Utang', 'Rp', '3', 'Juta', 'Bikin', 'Pegawai...","['Hidup', 'pria', 'bernama', 'Monika', 'Hutaur...","['utang', 'rp', '3', 'juta', 'bikin', 'pegawai...","['hidup', 'pria', 'bernama', 'monika', 'hutaur...","[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, nama, monika, hutauruk, 45, haru...","[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, nama, monika, hutauruk, 45, akhi..."


In [6]:
import stanza

try:
    stanza.download('id') 
except Exception as e:
    print(f"Error downloading Stanza model: {e}")
    print("Pastikan Anda memiliki koneksi internet atau model sudah terunduh sebelumnya.")

2025-05-24 16:32:14 INFO: Finished downloading models and saved to C:\Users\fairuuz\stanza_resources


In [11]:
try:
    nlp_id = stanza.Pipeline('id', processors='tokenize,pos', verbose=False)
except Exception as e:
    print(f"Error initializing Stanza pipeline: {e}")
    print("Pastikan model 'id' sudah terunduh dengan benar.")
    nlp_id = None 

In [ ]:
def pos_tag_stanza(token_list):
    """
    Melakukan POS tagging pada list token menggunakan Stanza.
    Input: list of words (tokens).
    Output: list of tuples (word, pos_tag).
    """
    if not nlp_id or not token_list or not isinstance(token_list, list):
        return []
    
    text = " ".join(token_list)
    if not text.strip(): 
        return []
        
    doc = nlp_id(text)
    pos_tags = []
    for sent in doc.sentences:
        for word in sent.words:
            pos_tags.append((word.text, word.upos)) 
    return pos_tags

In [13]:
if nlp_id:
    print("\nMemulai proses POS Tagging untuk kolom 'judul_stop_removed'...")
    df['judul_pos_tags'] = df['judul_stop_removed'].apply(pos_tag_stanza)
    print("POS Tagging untuk judul selesai.")

    print("\nMemulai proses POS Tagging untuk kolom 'isi_stop_removed'...")
    df['isi_pos_tags'] = df['isi_stop_removed'].apply(pos_tag_stanza)
    print("POS Tagging untuk isi berita selesai.")
else:
    print("\nStanza pipeline tidak terinisialisasi. Langkah POS tagging dilewati.")
    df['judul_pos_tags'] = pd.Series([[] for _ in range(len(df))])
    df['isi_pos_tags'] = pd.Series([[] for _ in range(len(df))])



Memulai proses POS Tagging untuk kolom 'judul_stop_removed'...
POS Tagging untuk judul selesai.

Memulai proses POS Tagging untuk kolom 'isi_stop_removed'...
POS Tagging untuk isi berita selesai.


In [14]:
print("\nDataFrame dengan kolom hasil POS Tagging (judul dan isi):")
print(df[['judul_stop_removed', 'judul_pos_tags', 'isi_stop_removed', 'isi_pos_tags']].head())


DataFrame dengan kolom hasil POS Tagging (judul dan isi):
                                  judul_stop_removed  \
0  [4, anak, bawah, umur, sangka, bunuh, palemban...   
1  [26, sangka, bekuk, lama, 3, bulan, akhir, kot...   
2  [kerja, kafe, siram, air, keras, hingga, wajah...   
3  [hilang, nyawa, pria, simalungun, gegara, rebu...   
4  [utang, rp, 3, juta, bikin, pegawai, akper, te...   

                                      judul_pos_tags  \
0  [(4, NUM), (anak, NOUN), (bawah, NOUN), (umur,...   
1  [(26, NUM), (sangka, NOUN), (bekuk, VERB), (la...   
2  [(kerja, NOUN), (kafe, NOUN), (siram, NOUN), (...   
3  [(hilang, VERB), (nyawa, NOUN), (pria, NOUN), ...   
4  [(utang, NOUN), (rp, PROPN), (3, NUM), (juta, ...   

                                    isi_stop_removed  \
0  [laku, bunuh, perkosa, aa, 14, orang, remaja, ...   
1  [waktu, kurang, lebih, 3, bulan, polres, probo...   
2  [orang, kerja, kafe, cengkareng, mas, 32, duga...   
3  [gegara, rebut, mikrofon, nyanyi, warung

In [15]:
df.head()

,url,judul,tanggal,isi_berita,panjang_judul,panjang_isi_berita,judul_rp,isi_rp,judul_tokens,isi_tokens,judul_tokens_lower,isi_tokens_lower,judul_stemmed,isi_stemmed,judul_stop_removed,isi_stop_removed,judul_pos_tags,isi_pos_tags
0,https://www.detik.com/sumbagsel/hukum-dan-krim...,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,2024-09-05 20:30:00,Pelaku pembunuhan dan pemerkosaan AA (14) seor...,73,2370,4 Anak di Bawah Umur Tersangka Pembunuhan di P...,Pelaku pembunuhan dan pemerkosaan AA 14 seoran...,"['4', 'Anak', 'di', 'Bawah', 'Umur', 'Tersangk...","['Pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","['4', 'anak', 'di', 'bawah', 'umur', 'tersangk...","['pelaku', 'pembunuhan', 'dan', 'pemerkosaan',...","[4, anak, di, bawah, umur, sangka, bunuh, di, ...","[laku, bunuh, dan, perkosa, aa, 14, orang, rem...","[4, anak, bawah, umur, sangka, bunuh, palemban...","[laku, bunuh, perkosa, aa, 14, orang, remaja, ...","[(4, NUM), (anak, NOUN), (bawah, NOUN), (umur,...","[(laku, NOUN), (bunuh, VERB), (perkosa, NOUN),..."
1,https://www.detik.com/jatim/hukum-dan-kriminal...,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,2024-09-05 18:38:00,"Dalam waktu kurang lebih 3 bulan, Polres Probo...",64,1764,26 Tersangka Dibekuk Selama 3 Bulan Terakhir d...,Dalam waktu kurang lebih 3 bulan Polres Probol...,"['26', 'Tersangka', 'Dibekuk', 'Selama', '3', ...","['Dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","['26', 'tersangka', 'dibekuk', 'selama', '3', ...","['dalam', 'waktu', 'kurang', 'lebih', '3', 'bu...","[26, sangka, bekuk, lama, 3, bulan, akhir, di,...","[dalam, waktu, kurang, lebih, 3, bulan, polres...","[26, sangka, bekuk, lama, 3, bulan, akhir, kot...","[waktu, kurang, lebih, 3, bulan, polres, probo...","[(26, NUM), (sangka, NOUN), (bekuk, VERB), (la...","[(waktu, NOUN), (kurang, ADV), (lebih, ADV), (..."
2,https://www.detik.com/sumut/hukum-dan-kriminal...,Pekerja Kafe Disiram Air Keras hingga Wajah 'B...,2024-09-04 21:40:00,"Seorang pekerja kafe di Cengkareng, MAS (32), ...",71,1995,Pekerja Kafe Disiram Air Keras hingga Wajah Be...,Seorang pekerja kafe di Cengkareng MAS 32 didu...,"['Pekerja', 'Kafe', 'Disiram', 'Air', 'Keras',...","['Seorang', 'pekerja', 'kafe', 'di', 'Cengkare...","['pekerja', 'kafe', 'disiram', 'air', 'keras',...","['seorang', 'pekerja', 'kafe', 'di', 'cengkare...","[kerja, kafe, siram, air, keras, hingga, wajah...","[orang, kerja, kafe, di, cengkareng, mas, 32, ...","[kerja, kafe, siram, air, keras, hingga, wajah...","[orang, kerja, kafe, cengkareng, mas, 32, duga...","[(kerja, NOUN), (kafe, NOUN), (siram, NOUN), (...","[(orang, NOUN), (kerja, NOUN), (kafe, NOUN), (..."
3,https://www.detik.com/sumut/hukum-dan-kriminal...,Hilang Nyawa Pria di Simalungun gegara Rebutan...,2024-09-03 09:03:00,Hanya gegara rebutan mikrofon untuk bernyanyi ...,70,1398,Hilang Nyawa Pria di Simalungun gegara Rebutan...,Hanya gegara rebutan mikrofon untuk bernyanyi ...,"['Hilang', 'Nyawa', 'Pria', 'di', 'Simalungun'...","['Hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","['hilang', 'nyawa', 'pria', 'di', 'simalungun'...","['hanya', 'gegara', 'rebutan', 'mikrofon', 'un...","[hilang, nyawa, pria, di, simalungun, gegara, ...","[hanya, gegara, rebut, mikrofon, untuk, nyanyi...","[hilang, nyawa, pria, simalungun, gegara, rebu...","[gegara, rebut, mikrofon, nyanyi, warung, tuak...","[(hilang, VERB), (nyawa, NOUN), (pria, NOUN), ...","[(gegara, NOUN), (rebut, VERB), (mikrofon, NOU..."
4,https://www.detik.com/sumut/hukum-dan-kriminal...,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,2024-09-03 08:01:00,Hidup pria bernama Monika Hutauruk (45) harus ...,68,3907,Utang Rp 3 Juta Bikin Pegawai Akper Tewas di T...,Hidup pria bernama Monika Hutauruk 45 harus be...,"['Utang', 'Rp', '3', 'Juta', 'Bikin', 'Pegawai...","['Hidup', 'pria', 'bernama', 'Monika', 'Hutaur...","['utang', 'rp', '3', 'juta', 'bikin', 'pegawai...","['hidup', 'pria', 'bernama', 'monika', 'hutaur...","[utang, rp, 3, juta, bikin, pegawai, akper, te...","[hidup, pria, nama, monika

In [ ]:
df.to_csv("../data/processed/preprocessed.csv", sep=',', index=False)

In [21]:
df_forlabelling = pd.read_csv("../data/processed/preprocessed.csv", usecols=["isi_pos_tags", "isi_stop_removed"])
df_forlabelling['id'] = ['D{:04d}'.format(i+1) for i in range(len(df_forlabelling))]

df_forlabelling = df_forlabelling[['id', 'isi_pos_tags', 'isi_stop_removed']]
df_forlabelling.head()

,id,isi_pos_tags,isi_stop_removed
0,D0001,"[('laku', 'NOUN'), ('bunuh', 'VERB'), ('perkos...","['laku', 'bunuh', 'perkosa', 'aa', '14', 'oran..."
1,D0002,"[('waktu', 'NOUN'), ('kurang', 'ADV'), ('lebih...","['waktu', 'kurang', 'lebih', '3', 'bulan', 'po..."
2,D0003,"[('orang', 'NOUN'), ('kerja', 'NOUN'), ('kafe'...","['orang', 'kerja', 'kafe', 'cengkareng', 'mas'..."
3,D0004,"[('gegara', 'NOUN'), ('rebut', 'VERB'), ('mikr...","['gegara', 'rebut', 'mikrofon', 'nyanyi', 'war..."
4,D0005,"[('hidup', 'NOUN'), ('pria', 'NOUN'), ('nama',...","['hidup', 'pria', 'nama', 'monika', 'hutauruk'..."


In [22]:
df_forlabelling.to_csv("../data/processed/forlabelling.csv", index=False)

## Prepare for Labelling

In [28]:
df = pd.read_csv('../data/processed/forlabelling.csv')

In [ ]:
result = pd.DataFrame(columns=['id', 'token', 'postag', 'entity'])

for i in range(len(df)):
    id = df['id'][i]
    print(id)
    list_pos_tags = eval(df['isi_pos_tags'][0])  

    for token_post in list_pos_tags:
        print(token_post)
        token= token_post[0]
        post = token_post[1]

        result = pd.concat([result, pd.DataFrame({'id': [id], 'token': [token], 'postag': [post], 'entity': [None]})], ignore_index=True)

result.to_csv('../data/processed/pseudolabelling.csv', index=False)

D0001
('laku', 'NOUN')
('bunuh', 'VERB')
('perkosa', 'NOUN')
('aa', 'PROPN')
('14', 'NUM')
('orang', 'NOUN')
('remaja', 'NOUN')
('putri', 'NOUN')
('kubur', 'NOUN')
('cina', 'PROPN')
('palembang', 'PROPN')
('tangkap', 'VERB')
('tetap', 'ADV')
('sangka', 'NOUN')
('empat', 'NUM')
('sangka', 'NOUN')
('rupa', 'NOUN')
('anak', 'NOUN')
('bawah', 'NOUN')
('umur', 'NOUN')
('sama', 'ADJ')
('korban', 'NOUN')
('hukum', 'NOUN')
('universitas', 'NOUN')
('taman', 'NOUN')
('siswa', 'NOUN')
('palembang', 'PROPN')
('dr', 'PROPN')
('azwar', 'PROPN')
('agus', 'NOUN')
('turut', 'ADV')
('prihatin', 'NOUN')
('jadi', 'VERB')
('bunuh', 'VERB')
('remaja', 'NOUN')
('putri', 'NOUN')
('laku', 'NOUN')
('remaja', 'NOUN')
('lakilaki', 'NOUN')
('usia', 'NOUN')
('baya', 'NOUN')
('korban', 'NOUN')
('jadi', 'VERB')
('sangat', 'ADV')
('prihatin', 'ADJ')
('lihat', 'VERB')
('aspek', 'NOUN')
('hukum', 'NOUN')
('jahat', 'ADJ')
('bendung', 'NOUN')
('iring', 'NOUN')
('kembang', 'NOUN')
('ilmu', 'NOUN')
('tahu', 'VERB')
('teknol

In [30]:
result = pd.read_csv('../data/processed/pseudolabelling.csv')
result.to_excel('../data/processed/pseudolabelling.xlsx', index=False)